In [3]:
""" Live Trainding with Python and Binance

Step 1) Request live data from Binance via API
Step 2) Data cleaning and Store it in SQLite
Step 3) Data Base update every second


"""

' Live Trainding with Python and Binance\n\nStep 1) Request live data from Binance via API\nStep 2) Data cleaning and Store it in SQLite\nStep 3) Data Base update every second\nStep 4) Get data from DB, apply Strategy and Excecute Order\nStep 5) Order Execution \n\n'

In [1]:
from binance.client import Client
import pandas as pd
import sqlalchemy
import asyncio
from binance import AsyncClient, BinanceSocketManager
import nest_asyncio
nest_asyncio.apply()


In [2]:
%run Keys.py
client = Client(api_key,secret_key)


In [3]:
# Request data using socket manager

bsm= BinanceSocketManager(client)
socket = bsm.trade_socket('BTCUSDT')

In [4]:

#* Step 2: Cleaning
def createframe(msg):
    """
    Transform the msg from the binance socket manager and transformer it to a dataframe
    Input:
            msg: form socket manager
    Ouput:
            df: Dataframe with symbol, Time and Price
    """
    df =pd.DataFrame([msg])
    df =df.loc[:,['s','E','p']]
    df.columns = ['symbol', 'Time','Price']
    df.Price =df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms') # readable time format
    return df

In [5]:
async def main(): #! Stop manuanly
    engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')
    client = await AsyncClient.create()
    
    bsm = BinanceSocketManager(client)  # set the socketManager #? for what is the user_timeout parametere?
    # start any sockets here, i.e a trade socket
    ts = bsm.trade_socket('BTCUSDT')  # pull data (candel, stick, price, etc)
   
    # then start receiving messages
    async with ts as tscm:
        while True:
            msg = await tscm.recv()
            #* Step 2: Cleaning
            if msg:
                frame = createframe(msg)
                #* Step 3: Update DB
                frame.to_sql('BTCUSDT',engine,if_exists ='append',index=False)
    await client.close_connection()

if __name__ == "__main__":

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())